<a href="https://colab.research.google.com/github/alexiusndoro/Python-Implied-Volatility-Calculator/blob/master/implied_volatility_calculator3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

In [0]:

import csv
import pandas as pd
import numpy as np
import scipy.stats as si
from math import log,sqrt,exp,pi


In [0]:
class DataReader:
  def __init__(self,file_name):
    self.file_name = str(file_name)
  
  def instantiate_trades(self):
    list_of_trades = []
    with open(self.file_name,'r') as csv_file:
      csv_reader = csv.reader(csv_file)

      #skips the first row as its the headings
      next(csv_reader)

      for line in csv_reader:
        
        each_trade = Trade(line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8])
        list_of_trades.append(each_trade)

      return list_of_trades

In [0]:
class Trade:
  def __init__(self,trade_id,underlying_type,underlying,r,T,K,option_type,model_type,market_price):
        self.trade_id = trade_id
        self.underlying_type = underlying_type
        self.underlying = float(underlying) 
        self.K=float(K)
        self.T= float(T)/365
        self.r = float (r)
        self.option_type = option_type
        self.model_type=model_type
        self.market_price = float(market_price)

In [0]:
class Model:
  
  #can improve this by just having black scholes and bachelier and having if statements to figure out which one
 

  @staticmethod
  def bs_put(Trade,sigma):
      S, K, r, T = Trade.underlying, Trade.K, Trade.r,Trade.T

      d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
      d2 = d1 - sigma * sqrt(T)

      bs_put_price = K * exp(-r * T) * si.norm.cdf(-d2) -S * si.norm.cdf(-d1)  
      return bs_put_price

  
  @staticmethod
  def bs_call(Trade,sigma):
      S, K, r, T = Trade.underlying, Trade.K, Trade.r, Trade.T

      d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
      d2 = d1 - sigma * sqrt(T)

      bs_call_price = S * si.norm.cdf(d1) - K * exp(-r * T) * si.norm.cdf(d2)

      return bs_call_price

  @staticmethod
  def ba_call(Trade,sigma):
      S, K, r, T = Trade.underlying, Trade.K, Trade.r,Trade.T
      d1  = (S-K)/(sigma*sqrt(T))

      ba_call_price =   ((S - K)*si.norm.cdf(d1) + sigma*sqrt(T)*si.norm.pdf(d1))*exp(-r*T)
      return ba_call_price
 
  @staticmethod
  def ba_put(Trade,sigma):
      S, K, r,T = Trade.underlying, Trade.K, Trade.r, Trade.T
      d1  = (S-K)/(sigma*sqrt(T))

      ba_put_price = ((K-S)*si.norm.cdf(-d1) + sigma*sqrt(T)*si.norm.pdf(d1)) * exp(-r*T)
      return ba_put_price



In [0]:
class NewtonSolver: 
   
  @staticmethod
  def calculate_volatility(Trade,vol):
    #note here that the parameter vol is the initial guess for volatility

      S, K, r,T, C = Trade.underlying, Trade.K, Trade.r,  Trade.T, Trade.market_price
     

      if Trade.model_type== "BlackScholes": 
        d1 = (log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * sqrt(T))
      elif Trade.model_type== "Bachelier": 
         d1  = (S-K)/(vol*sqrt(T))

      #tolerances
      tolerance = 1e-8
      epsilon =1

      count =0
      max_iteration = 1000
      

      while epsilon > tolerance:
        count+=1
        if count>= max_iteration:
          print('breaking on count')
          return np.nan

        initital_vol = vol

        #funtion_value is the funtion we are finding the root of
        function_value=0   
           
        if Trade.model_type == "BlackScholes":   
          if Trade.option_type == "Call":
            function_value = Model.bs_call(Trade,vol) - C
          if Trade.option_type == "Put":
            function_value = Model.bs_put(Trade,vol) - C
        if Trade.model_type == "Bachelier":
          if Trade.option_type == "Call":
             function_value = Model.ba_call(Trade,vol) - C
          if Trade.option_type == "Put":
              function_value = Model.ba_put(Trade,vol) - C  

        vega = S*si.norm.pdf(d1)*sqrt(T)

        vol = -function_value/vega +vol  

        epsilon = abs((vol-initial_vol)/initial_vol)

        vega = (1 / sqrt(2 * pi)) * S * sqrt(T) * exp(-(si.norm.cdf(d1) ** 2) * 0.5)
      #print("number of iterations is", count)
   
      return vol

In [0]:
class OutputWriter:

    @staticmethod
    def write_output_file(trades_list,file_name):
      
      with open(str(file_name),'w') as new_file:
        csv_writer=csv.writer(new_file)
        
        csv_writer.writerow(["trade_id","model","option_type","implied_volatility"])
        
        for trade in trades_list:
          
          
          implied_volatility = newton_solver.calculate_volatility(trade,0.25)
          csv_writer.writerow([trade.trade_id,trade.model_type,trade.option_type,implied_volatility])


In [0]:
class IVCRunner:

  @staticmethod
  def run_program():
    dr = DataReader("input.csv")
   
    trades_list  = dr.instantiate_trades()
   
    OutputWriter.write_output_file(trades_list[0:20],'volatilities.csv')




In [53]:
IVCRunner.run_program()

[<__main__.Trade object at 0x7f5534644f28>, <__main__.Trade object at 0x7f5534644278>, <__main__.Trade object at 0x7f5534644198>, <__main__.Trade object at 0x7f5534644160>, <__main__.Trade object at 0x7f5534644a20>, <__main__.Trade object at 0x7f55346447b8>, <__main__.Trade object at 0x7f5534644c88>, <__main__.Trade object at 0x7f5534637fd0>, <__main__.Trade object at 0x7f55345d2dd8>, <__main__.Trade object at 0x7f55345d2710>, <__main__.Trade object at 0x7f55345d2a20>, <__main__.Trade object at 0x7f55345d2828>, <__main__.Trade object at 0x7f55345d2eb8>, <__main__.Trade object at 0x7f55345d20f0>, <__main__.Trade object at 0x7f55345d2080>, <__main__.Trade object at 0x7f55345d2358>, <__main__.Trade object at 0x7f55345d2588>, <__main__.Trade object at 0x7f55345d21d0>, <__main__.Trade object at 0x7f55345d2048>, <__main__.Trade object at 0x7f55345d2908>, <__main__.Trade object at 0x7f5534624048>, <__main__.Trade object at 0x7f5534624a20>, <__main__.Trade object at 0x7f5534624390>, <__main__.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars


breaking on count
breaking on count
breaking on count
breaking on count
